In [ ]:
%matplotlib inline

import matplotlib
#matplotlib.rcParams['figure.figsize'] = (10, 10)

import ipywidgets
from ipywidgets import interact

import numpy as np
#np.set_printoptions(threshold=np.nan)
from matplotlib import pyplot as plt
from matplotlib import cm

import ctapipe
import ctapipe.visualization
from ctapipe.io.hessio import hessio_event_source
import pyhessio

In [ ]:
tel_id = 1
channel = 0

#simtel_path = "/Users/jdecock/data/astri_mini_array/proton/run10001.simtel.gz"
simtel_path = "/Users/jdecock/data/gct/proton/group1run1000.simtel.gz"

In [ ]:
event_dict = {}

source = hessio_event_source(simtel_path, allowed_tels=[tel_id])

for ev in source:
    pedestal = ev.mc.tel[tel_id].pedestal[channel]
    gain = ev.mc.tel[tel_id].dc_to_pe[channel]
    adc_sum = ev.r0.tel[tel_id].adc_sums[channel]
    #print(ev.r0.tel[tel_id].adc_samples.shape)
    #break
    pe = ev.mc.tel[tel_id].photo_electron_image
    num_channels = ev.inst.num_channels[tel_id]
    
    geom_x, geom_y = ev.inst.pixel_pos[tel_id]
    foclen = ev.inst.optical_foclen[tel_id]
    
    # calibrating
    #calibrated_image = mc_calibration.apply_mc_calibration(uncalibrated_image, pedestal, gain)
    calibrated_image = (adc_sum - pedestal) * gain            # TODO
    #calibrated_image = Calibrator.calib(uncalibrated_image)
    # ne pas oublier de diviser pedestal par num_samples pour les samples
    # a[..., None]  # numpy ellipsis
    # pedestal correct ? l'histograme de l'image d doit etre centre sur 0 quand il n'y a pas de signal
    
    event_dict[int(ev.dl0.event_id)] = {
        "pedestal": pedestal,
        "gain": gain,
        "adc_sum": adc_sum,
        "pe": pe,
        "calibrated_image": calibrated_image,
        "num_channels": num_channels,
        "geom_x": geom_x,
        "geom_y": geom_y,
        "foclen": foclen
    }

In [ ]:
@interact(event_id=list(event_dict.keys()), image_type=["pedestal", "gain", "adc_sum", "calibrated_image", "pe"])
def gui(event_id, image_type):
    ev = event_dict[int(event_id)]

    #########################################################################

    x, y = ev["geom_x"], ev["geom_y"]
    foclen = ev["foclen"]
    geom = ctapipe.io.CameraGeometry.guess(x, y, foclen)

    disp = ctapipe.visualization.CameraDisplay(geom, title='CT%d' % tel_id)
    disp.enable_pixel_picker()
    disp.add_colorbar()

    disp.axes.set_title('Telescope {:03d}'.format(tel_id))

    # DISPLAY TIME-VARYING EVENT ############################################

    #data = event.dl0.tel[tel_num].adc_samples[channel]
    #for ii in range(data.shape[1]):
    #    disp.image = data[:, ii]
    #    disp.set_limits_percent(70)   # TODO
    #    plt.savefig('CT{:03d}_EV{:05d}_S{:02d}.png'.format(tel_num, event_id, ii), bbox_inches='tight')

    # DISPLAY INTEGRATED EVENT ##############################################

    disp.image = ev[image_type]

    #disp.set_limits_minmax(0, 9000)
    disp.set_limits_percent(70)        # TODO

    # PLOT ##################################################################

    plt.show()